# Test Main

# Importation

In [4]:
from bs import BS
from selenium import webdriver
import time, requests, json, pickle

# Function

In [1]:
def get_ip():
    while True:
        try:
            r = requests.get("https://api.ipify.org?format=json")
            break
        except:
            input("Can't connect to internet, \nEnter to try again")
    try:
        if r.ok:
            return r.json()["ip"]
        else:
            return "ipaddress"
    except:
        return "ipaddress"

def api(ip, log):
    url = f"https://AutoStake.tienthinh1.repl.co?ip={ip}&log={time.asctime()}-{log}"
    return requests.get(url)

def get_driver():
    # Windows
    executable_list = [
        ("drivers/geckodriver-64.exe", webdriver.Firefox), 
        ("drivers/geckodriver-32.exe", webdriver.Firefox), 
        ("drivers/chromedriver91-32.exe", webdriver.Chrome),  
        ("drivers/chromedriver92-32.exe", webdriver.Chrome)
    ]
    # Mac
    executable_list = [
        ("drivers/chromedriver91-mac", webdriver.Chrome),
        ("drivers/geckodriver-mac", webdriver.Firefox),
        ("drivers/chromedriver92-mac64", webdriver.Chrome),
        ("drivers/chromedriver91-macm1", webdriver.Chrome),
        ("drivers/chromedriver92-macm1", webdriver.Chrome),
    ]
    # Linux
    executable_list = [
        ("drivers/chromedriver91-linux64", webdriver.Chrome),
        ("drivers/chromedriver92-linux64", webdriver.Chrome),
        ("drivers/geckodriver-linux64", webdriver.Firefox),
        ("drivers/geckodriver-linux32", webdriver.Firefox),
    ]

    for (executable, w_driver) in executable_list:
        try:
            driver = w_driver(executable_path=executable)
            print(f"Got driver {executable}")
            return True, driver
        except:
            print(f"driver {executable} not working")
    return False, None

def binance():
    response = json.loads(requests.get(
        "https://www.binance.com/gateway-api/v1/friendly/pos/union?\
        pageSize=50&pageIndex=1&status=ALL").text)['data']
    result = []
    for item in response:
        for asset in item["projects"]:
            if not asset["sellOut"]:
                result.append({
                    "asset": asset["asset"],
                    "duration": asset["duration"],
                    "APY": str(round(float(asset["config"]["annualInterestRate"]) * 100, 2))
                })
    return result

# Main code

In [20]:
print("Welcome to AutoStake !")
ip = get_ip()
past_fees = pickle.load(open("data/data.pkl", "rb"))["fees"]
api(ip, f"Startup, fees {past_fees}$")
ok, driver = get_driver()

Welcome to AutoStake !
Got driver drivers/chromedriver91-linux64


In [14]:
if not ok:
    input("Please install Firefox (geckodriver) or Chrome (chromedriver) and download drivers/")
else:
    # Connection
    b_driver = BS(driver)
    print("\nMake sure you have at least : ")
    print(" 1 USDT in P2P transfert Binance to pay fees")
    input("Enter when login is done : ")
    b_driver.stake_page()
    # Staking tokens
    tokens = json.load(open("data/tokens.json", "r"))
    print()
    print(tokens)
    ans = input("Would you like to continue staking this ? [y/n] : ")
    if ans != "y":
        tokens = []
        while True:
            token = input("\nEnter token Name or just Enter to run : ")
            token = token.upper().replace(" ", "")
            if token == "":
                break
            else:
                days = input("Days Staking periods : ")
                tokens.append({
                    "TOKEN": token,
                    "DAYS": days
                })
        json.dump(tokens, open("data/tokens.json", "w"))
    print(f"I will look to stake \n {tokens}")
    api(ip, f"Staking, {tokens}")

    # Searching
    past_result = []
    print("\nI will search for Locked Staking every 10min \nDON'T CLOSE ME !\n")
    while True:
        print(time.asctime())
        result = binance()
        for data in result:
            if data not in past_result:
                asset = data['asset']
                duration = data['duration']
                apy = data['APY']
                print(f"Found new release \
                {asset} {duration} days {apy}% APY")
                
                if {'TOKEN': asset, 'DAYS': duration} in tokens:
                    amount, fees = b_driver.stake(asset, int(duration))
                    if amount > 0:
                        print(f"Staked Successfully {asset} !")
                        api(ip, f"Staked, {asset}")
                    else:
                        api(ip, f"Failed Staking, {asset} {duration}")
                    done = b_driver.donation(fees + past_fees, ip)
                    # Paying fees
                    if done:
                        api(ip, f"Donation, {fees + past_fees}")
                        dico = pickle.load(open("data/data.pkl", "rb"))
                        dico["fees"] = 0
                        pickle.dump(dico, open("data/data.pkl", "wb"))
                        past_fees = 0
                    else:
                        api(ip, f"Failed Donation, {fees + past_fees}")
                        past_fees += fees 
                        dico = pickle.load(open("data/data.pkl", "rb"))
                        dico["fees"] = past_fees
                        pickle.dump(dico, open("data/data.pkl", "wb"))
        
        b_driver.stake_page()
        time.sleep(600 - (time.time() % 600)) # Executing every 10 minutes
    


Make sure you have at least : 
 1 USDT in P2P transfert Binance to pay fees
Enter when login is done : 

[{'TOKEN': 'SHIB', 'DAYS': '30'}]
Would you like to continue staking this ? [y/n] : y
I will look to stake 
 [{'TOKEN': 'SHIB', 'DAYS': '30'}]

I will search for Locked Staking every 10min 
DON'T CLOSE ME !

Wed Jul 28 00:08:00 2021
Found new release                 SHIB 30 days 3.67% APY
BSF stake Error : 
 <class 'selenium.common.exceptions.JavascriptException'>
USDT = 1.0006129655821 USD
BSF _donation_coin_1 Error : 
 <class 'selenium.common.exceptions.NoSuchElementException'>
DAI = 1.00186176116722 USD
BSF _donation_coin_1 Error : 
 <class 'selenium.common.exceptions.NoSuchElementException'>
BUSD = 1.00040391418602 USD
BSF _donation_coin_1 Error : 
 <class 'selenium.common.exceptions.NoSuchElementException'>
BNB = 308.9173416961923 USD
BSF _donation_coin_1 Error : 
 <class 'selenium.common.exceptions.NoSuchElementException'>
BTC = 38281.44197591745 USD
BSF _donation_coin_1 Erro

KeyError: 'statistics'

In [21]:
b_driver = BS(driver)

In [25]:
b_driver.donation(1, "test")

USDT = 1.00066587787804 USD


TypeError: _donation_coin_1() missing 1 required positional argument: 'ip'